# Training an SNN using Neuroevolution!

Cartpole

In [1]:
import spyx
import spyx.nn as snn

# JAX imports
import os
import jax
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".70"
from jax import numpy as jnp

from tqdm import tqdm

# implement our SNN in DeepMind's Haiku
import haiku as hk

# optimize the parameters using evosax
import evosax
from evosax.strategies import OpenES as ES
from evosax.strategies import LM_MA_ES as LMMAES

import gymnax

# rendering tools
import matplotlib.pyplot as plt
%matplotlib notebook
import graphviz
import mediapy as media

/home/legion/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/legion/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Create Env

In [2]:
rng = jax.random.PRNGKey(0)
rng, key_reset = jax.random.split(rng, 2)

# Instantiate the environment & its settings.
env, env_params = gymnax.make("CartPole-v1")

# Reset the environment.
obs, _ = env.reset(key_reset, env_params)

In [3]:
class binarize:
    def __init__(self, neuron_count, min_val, max_val):
        self.neuron_count = neuron_count
        self.min_val = min_val
        self.max_val = max_val
        
    def __call__(self, obs):
        digital = jnp.digitize(obs, jnp.linspace(self.min_val, self.max_val, self.neuron_count))
        return jax.nn.one_hot(digital, self.neuron_count)
    
class NeuromorphicCartpole:
    def __init__(self, angle_neurons=16, cart_v_neurons=16, pole_w_neurons=16):
        self.angle_converter = binarize(angle_neurons, -.21, .21)
        self.v_converter = binarize(cart_v_neurons, -3.5, 3.5)
        self.w_converter = binarize(pole_w_neurons, -3.5, 3.5)
        
    def __call__(self, obs):
        cart_v = jnp.array([obs[1]]) #self.v_converter(obs[1])
        theta = self.angle_converter(obs[2])
        pole_w = jnp.array([obs[3]]) #self.w_converter(obs[3])
        
        return jnp.concatenate([cart_v, theta, pole_w])
        

In [4]:
adapter = NeuromorphicCartpole()

## SNN

In [5]:
    
def action_selection(spike_trains):
    return jnp.argmax(spike_trains, axis=0)
        
def controller(x, state):
    # seqs is [T, F].
    core = hk.DeepRNN([
        hk.Linear(64, with_bias=False),
        snn.LIF(64, beta=0.8, activation=spyx.activation.Heaviside()),
        hk.Linear(2, with_bias=False),
        snn.LI(2)
    ])
    
    spikes, out_state = core(x, state)
    return spikes, out_state

In [6]:
key = jax.random.PRNGKey(0)
init_state = (jnp.zeros(64, dtype=jnp.float16), jnp.zeros(2, dtype=jnp.float16))
policy = hk.without_apply_rng(hk.transform(controller))
policy_params = policy.init(rng=key, x=adapter(obs), state=init_state)

In [7]:
policy.apply(policy_params, adapter(obs), init_state)

(Array([0., 0.], dtype=float16),
 (Array([ 0.04645 , -0.1208  ,  0.433   ,  0.0579  ,  0.3577  ,  0.185   ,
         -0.05032 , -0.3591  , -0.0857  , -0.238   ,  0.01361 ,  0.0888  ,
          0.007725, -0.1672  , -0.0688  ,  0.1993  , -0.1892  ,  0.04123 ,
          0.2935  , -0.03607 , -0.3706  , -0.151   ,  0.2104  , -0.05933 ,
          0.2607  ,  0.2942  ,  0.169   , -0.3967  , -0.1824  , -0.01935 ,
         -0.4172  ,  0.4382  , -0.1425  ,  0.01437 ,  0.1927  ,  0.264   ,
          0.03183 , -0.1364  , -0.03696 ,  0.3386  ,  0.163   ,  0.2075  ,
          0.0618  , -0.086   ,  0.343   , -0.09375 , -0.3103  ,  0.304   ,
          0.3247  , -0.1263  , -0.06696 , -0.2817  , -0.03186 ,  0.3115  ,
          0.00876 , -0.4177  , -0.07947 , -0.2014  , -0.2556  ,  0.1333  ,
          0.2598  , -0.0686  , -0.29    ,  0.3643  ], dtype=float16),
  Array([0., 0.], dtype=float16)))

In [8]:
adapter(obs).shape

(18,)

## Evolution

In [9]:
# Instantiate the environment & its settings.
env, env_params = gymnax.make("CartPole-v1")

def rollout(policy_params, init_policy_state, env_params, rng_input, steps_in_episode):
    """Rollout a jitted gymnax episode with lax.scan."""
    # Reset the environment
    rng_reset, rng_episode = jax.random.split(rng_input)
    obs, env_state = env.reset(rng_reset, env_params)

    def policy_step(state_input, tmp):
        """lax.scan compatible step transition in jax env."""
        obs, env_state, policy_params, policy_state, rng = state_input
        rng, rng_step, rng_net = jax.random.split(rng, 3)
        activation, new_policy_state = policy.apply(policy_params, adapter(obs), policy_state)
        action = action_selection(activation)
        next_obs, next_state, reward, done, _ = env.step(
            rng_step, env_state, action, env_params
        )
        carry = [next_obs, next_state, policy_params, new_policy_state, rng]
        return carry, [obs, action, reward, next_obs, done]

    # Scan over episode step loop
    _, scan_out = jax.lax.scan(
        policy_step,
        [obs, env_state, policy_params, init_policy_state, rng_episode],
        (),
        steps_in_episode
    )
    # Return masked sum of rewards accumulated by agent in episode
    obs, action, reward, next_obs, done = scan_out
    return obs, action, reward, next_obs, done

jit_rollout = jax.jit(rollout, static_argnums=[3,4])
vector_rollout = jax.vmap(jit_rollout, (0,None,None,None,None))

In [163]:
def evolution(SNN, params, epochs=50, trials=128, steps=250, key=0):

    rng = jax.random.PRNGKey(key)
    param_reshaper = evosax.ParameterReshaper(params)
    
    # Instantiate and initialize the evolution strategy
    strategy = ES(popsize=128,
                      num_dims=param_reshaper.total_params,
                      opt_name="adam"
                    )

    es_params = strategy.default_params
    es_params = es_params.replace(sigma_init=0.1, sigma_decay=0.999, sigma_limit=0.01)
    es_params = es_params.replace(opt_params=es_params.opt_params.replace(
        lrate_init=0.1, lrate_decay=0.999, lrate_limit=0.001))
    
    # check the initialization here....
    strat_state = strategy.initialize(rng, es_params)
        
    @jax.jit
    def step(rng, state):
        rng, rng_ask, rng_eval = jax.random.split(rng, 3)
        # ASK
        pop, state = strategy.ask(rng_ask, state)
        population_params = param_reshaper.reshape(pop)
        init_policy_state = init_state
                
        # EVAL
        obs, action, reward, next_obs, done = \
            vector_rollout(population_params, init_policy_state, env_params, rng_eval, steps)
        

        # TELL
        total_reward = jnp.sum(reward*(1-done), axis=-1)        
        
        return rng, pop, total_reward
    
    
    
    for gen in range(epochs):
        
        # figure out way to JIT this inner loop better to account for trials
        total_reward = jnp.zeros([128])
        
        pbar = tqdm([*range(trials)])
        pbar.set_description("Epoch #{}".format(gen))
        for trials_so_far in pbar:
            rng, pop, reward = step(rng, strat_state)
            total_reward += reward
            pbar.set_postfix(Reward=jnp.max(total_reward)/trials_so_far)
        
        strat_state = strategy.tell(pop, -total_reward/trials, strat_state)
            
        
    elite = param_reshaper.reshape(jnp.array([strat_state.best_member]))
    return jax.tree_util.tree_map(jnp.squeeze, elite)

In [164]:
elite_params = evolution(policy, policy_params)

ParameterReshaper: 1280 parameters detected for optimization.


Epoch #0:   2%|▎              | 3/128 [00:00<00:26,  4.80it/s, Reward=309.33334]

[228. 223. 231. 225. 228. 230. 226. 227. 227. 233. 226. 224. 225. 225.
 231. 226. 224. 228. 224. 224. 224. 225. 226. 225. 236. 224. 224. 224.
 224. 224. 231. 223. 229. 224. 225. 225. 226. 228. 232. 224. 224. 226.
 230. 224. 224. 224. 226. 224. 225. 226. 227. 226. 225. 228. 224. 224.
 229. 231. 224. 226. 226. 226. 225. 229. 226. 226. 228. 225. 226. 234.
 224. 224. 225. 225. 229. 225. 225. 227. 225. 226. 228. 229. 233. 223.
 224. 224. 225. 225. 225. 223. 229. 224. 223. 226. 223. 224. 225. 224.
 226. 224. 235. 226. 224. 225. 224. 226. 226. 227. 224. 224. 224. 224.
 224. 224. 225. 226. 226. 230. 224. 224. 227. 229. 224. 224. 228. 224.
 224. 224.]
[224. 231. 229. 228. 225. 235. 224. 225. 225. 228. 225. 227. 224. 227.
 227. 224. 224. 228. 224. 224. 224. 227. 224. 235. 224. 227. 226. 226.
 225. 224. 225. 232. 231. 228. 225. 224. 224. 229. 224. 231. 224. 228.
 227. 225. 227. 225. 227. 224. 230. 224. 226. 224. 224. 227. 230. 232.
 225. 228. 224. 226. 225. 225. 229. 224. 225. 237. 224. 227. 225.

Epoch #0:   5%|▊              | 7/128 [00:01<00:12,  9.68it/s, Reward=262.14288]

[224. 224. 231. 224. 229. 227. 225. 227. 224. 224. 225. 224. 226. 224.
 231. 230. 224. 227. 228. 227. 224. 224. 225. 225. 227. 224. 224. 226.
 224. 224. 225. 224. 228. 225. 227. 227. 230. 224. 224. 229. 231. 226.
 232. 224. 225. 224. 226. 226. 226. 224. 224. 226. 224. 229. 225. 224.
 224. 224. 225. 240. 224. 224. 224. 224. 224. 223. 227. 224. 226. 224.
 225. 230. 229. 227. 226. 224. 224. 225. 232. 224. 224. 225. 233. 225.
 224. 226. 230. 224. 231. 225. 224. 229. 224. 224. 234. 226. 224. 227.
 231. 224. 225. 224. 226. 234. 225. 225. 225. 229. 224. 224. 230. 224.
 225. 224. 224. 228. 224. 225. 227. 231. 225. 224. 224. 227. 224. 225.
 224. 224.]
[225. 224. 224. 224. 233. 225. 225. 224. 224. 228. 224. 225. 226. 224.
 226. 224. 225. 224. 225. 224. 224. 224. 224. 237. 224. 229. 224. 225.
 238. 224. 225. 224. 228. 230. 228. 224. 224. 225. 227. 231. 230. 224.
 229. 227. 231. 227. 228. 224. 229. 224. 225. 225. 224. 239. 228. 226.
 230. 226. 231. 232. 224. 231. 224. 224. 225. 228. 224. 224. 234.

...

[235. 233. 231. 236. 236. 229. 237. 235. 231. 229. 233. 238. 234. 235.
 234. 229. 235. 225. 230. 237. 228. 229. 232. 234. 236. 230. 226. 236.
 229. 235. 232. 235. 235. 237. 231. 232. 227. 229. 236. 240. 226. 234.
 238. 235. 234. 230. 231. 231. 236. 233. 224. 232. 231. 233. 231. 233.
 226. 241. 236. 225. 233. 225. 233. 230. 228. 231. 231. 225. 231. 234.
 227. 231. 231. 238. 233. 225. 236. 224. 230. 236. 229. 236. 234. 231.
 239. 231. 232. 233. 225. 233. 239. 230. 235. 233. 231. 224. 229. 235.
 232. 230. 241. 233. 231. 225. 236. 226. 226. 228. 232. 235. 232. 235.
 232. 237. 238. 231. 239. 232. 229. 228. 230. 225. 226. 236. 230. 237.
 227. 236.]


Epoch #19:  80%|█████████▌  | 102/128 [00:06<00:01, 15.90it/s, Reward=235.89108]

[237. 234. 232. 227. 228. 239. 232. 237. 234. 239. 239. 238. 234. 230.
 225. 237. 236. 239. 233. 233. 234. 235. 228. 239. 234. 235. 236. 228.
 240. 240. 226. 234. 231. 229. 234. 236. 235. 238. 237. 228. 238. 230.
 236. 235. 237. 226. 231. 226. 238. 233. 237. 237. 233. 233. 230. 227.
 230. 227. 233. 231. 232. 239. 228. 238. 227. 232. 234. 238. 237. 228.
 235. 228. 233. 232. 225. 234. 232. 236. 243. 229. 234. 229. 236. 237.
 239. 235. 240. 234. 235. 229. 230. 239. 226. 228. 236. 235. 236. 235.
 228. 233. 231. 234. 231. 236. 226. 237. 231. 229. 232. 240. 231. 235.
 228. 232. 231. 228. 235. 236. 240. 241. 237. 234. 233. 238. 234. 229.
 233. 228.]


Epoch #19:  80%|█████████▌  | 102/128 [00:06<00:01, 15.88it/s, Reward=235.89108]


KeyboardInterrupt: 

In [162]:
#activation_seq = []
action_seq = []
state_seq, reward_seq = [], []
rng, rng_reset = jax.random.split(rng)
obs, env_state = env.reset(rng_reset, env_params)
new_policy_state = init_state
while True:
    state_seq.append(env_state)
    rng, rng_step = jax.random.split(rng, 2)
    activation, new_policy_state = policy.apply(elite_params, adapter(obs), new_policy_state)
    action = action_selection(activation)
    action_seq.append(action)
    #activation_seq.append(activation)
    next_obs, next_env_state, reward, done, info = env.step(
        rng_step, env_state, action, env_params
    )
    reward_seq.append(reward)
    if done:
        break
    else:
        obs = next_obs
        env_state = next_env_state

cum_rewards = jnp.cumsum(jnp.array(reward_seq))
len(action_seq)

26

In [165]:
action_seq

[Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(0, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32),
 Array(1, dtype=int32)]

Something might be wrong with rng...